# Réseaux Navigocorpus

In [7]:
import networkx as nx
from poitousprint import Portic
from ipysigma import Sigma

In [25]:
INNER_ADMIRALTIES = {'La Rochelle', 'Marennes', 'Sables-d’Olonne'}

In [2]:
client = Portic()

In [9]:
flows = client.get_flows(year=1789, source_subset='Poitou_1789')
flows = [flow for flow in flows if flow['departure_action'] == 'Out']

## Réseau macro

In [60]:
inner_graph = nx.DiGraph()
macro_graph = nx.DiGraph()

def add_node(g, name, admiralty=None, peche=0):
    if admiralty is None:
        admiralty = 'n/a'
    
    if g.has_node(name):
        g.nodes[name]['peche'] += peche
    else:
        g.add_node(
            name,
            admiralty=admiralty,
            peche=peche,
            in_region=admiralty in INNER_ADMIRALTIES,
            inside_degree=0,
            outside_degree=0
        )

def add_edge(g, source, target, tonnage):
    if g.has_edge(source, target):
        attr = g[source][target]
        attr['weight'] += 1
        attr['tonnage'] += tonnage
    else:
        g.add_edge(
            source,
            target,
            weight=1,
            tonnage=tonnage
        )

for flow in flows:
    source = flow['departure']
    target = flow['destination']
    
    source_admiralty = flow['departure_admiralty']
    target_admiralty = flow['destination_admiralty']
    
    tonnage = 0

    try:
        tonnage = int(flow['tonnage'])
    except ValueError:
        pass
    
    inside_flow = source_admiralty in INNER_ADMIRALTIES and target_admiralty in INNER_ADMIRALTIES
    
    # Macro graph
    if source == target:
        add_node(macro_graph, source, source_admiralty, 1)
    else:
        add_node(macro_graph, source, source_admiralty)
        add_node(macro_graph, target, target_admiralty)
        add_edge(macro_graph, source, target, tonnage)
        
        if source_admiralty in INNER_ADMIRALTIES:
            if inside_flow:
                macro_graph.nodes[source]['inside_degree'] += 1
            else:
                macro_graph.nodes[source]['outside_degree'] += 1

In [61]:
nx.write_gexf(macro_graph, 'macro.gexf')